In [159]:
df_monthly = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df_monthly = df_monthly.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df_monthly.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df_monthly.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df_monthly.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


df_MC = df_numfirm.multiply(df_avgsize, axis=0)
df_MC = df_MC.loc[df_48ind.index]


df_BtoM = df_monthly.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

df_48ind = df_48ind.loc[df_mom.index]
df_MC = df_MC.loc[df_mom.index]
df_BtoM = df_BtoM.loc[df_mom.index]

df_ret_shift = df_48ind.shift(-1)

############ 5 factors    ##############

df_FF_5factors = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_5_Factors_2x3.CSV', header=2)
df_FF_5factors = df_FF_5factors.rename(columns={'Unnamed: 0': 'Date'})
df_FF_5 = df_FF_5factors.iloc[:727].copy()
df_FF_5['Date'] = pd.to_datetime(df_FF_5['Date'], format='%Y%m')
df_FF_5.set_index('Date', inplace=True)
df_FF_5 = df_FF_5.apply(pd.to_numeric, errors='coerce')
df_FF_5 = df_FF_5.loc[df_mom.index]

df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)
df_daily = df_daily.rename(columns={'Unnamed: 0': 'Date'})

df_daily_ret = df_daily.iloc[:25670].copy()
df_daily_ret['Date'] = pd.to_datetime(df_daily_ret['Date'], format='%Y%m%d')
df_daily_ret.set_index('Date', inplace=True)
df_daily_ret = df_daily_ret.apply(pd.to_numeric, errors='coerce')
df_daily_ret.replace(-99.99, np.nan, inplace = True)
df_daily_ret.replace(-999, np.nan, inplace = True)
df_daily_ret.dropna(inplace = True)

df_daily_ret_reshaped = df_daily_ret.iloc[231:].copy()

########## 3 factors ########## 

df_FF_3 = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_Factors_daily.CSV', header=3)
df_FF_3 = df_FF_3.rename(columns={'Unnamed: 0': 'Date'})
df_FF_daily = df_FF_3.iloc[:25670].copy()
df_FF_daily['Date'] = pd.to_datetime(df_FF_daily['Date'], format='%Y%m%d')
df_FF_daily.set_index('Date', inplace=True)
df_FF_daily = df_FF_daily.apply(pd.to_numeric, errors='coerce')
df_FF_daily.replace(-99.99, np.nan, inplace = True)
df_FF_daily.replace(-999, np.nan, inplace = True)
df_FF_daily.dropna(inplace = True)
df_FF_daily = df_FF_daily.loc[df_daily_ret.index]

rf = df_FF_daily['RF'].mean()

######################################## 1.1 ########################################

def calculate_betas(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']

    betas = []  # Create an empty list to store the betas

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = Rm_t

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        betas.append(results.params[1])  # Add the betas to the list

    # Convert the list of betas to a numpy array
    betas = np.array(betas)

    return betas


# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the betas for each month
monthly_betas = {}
for date in date_range:
    start_date = date - pd.DateOffset(months=12)
    end_date = date
    betas = calculate_betas(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_betas[date] = betas
    
    # Convert the dictionary to a DataFrame
df_monthly_betas = pd.DataFrame(monthly_betas.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_betas['Betas'] = df_monthly_betas['Betas'].apply(list)

# Set the Date column as the index
df_monthly_betas.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_betas = df_monthly_betas['Betas'].apply(pd.Series)

# Set the index of df_monthly_betas to match df_48ind
df_monthly_betas.index = df_48ind.index

# Set the column names of df_monthly_betas to match df_48ind
df_monthly_betas.columns = df_48ind.columns

def calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']
    SMB = df_FF_daily_selected['SMB']
    HML = df_FF_daily_selected['HML']

    volatilities = []  # Create an empty list to store the volatilities

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = pd.concat([Rm_t, SMB, HML], axis=1)

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        residuals = results.resid
        volatility = np.std(residuals)

        volatilities.append(volatility)  # Add the volatility to the list

    # Convert the list of volatilities to a numpy array
    volatilities = np.array(volatilities)

    return volatilities


# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the volatilities for each month
monthly_volatilities = {}
for date in date_range:
    start_date = date  - pd.DateOffset(months=1) + pd.DateOffset(days=1)
    end_date = date
    volatilities = calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_volatilities[date] = volatilities
    
        # Convert the dictionary to a DataFrame
df_monthly_vol = pd.DataFrame(monthly_volatilities.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_vol['Betas'] = df_monthly_vol['Betas'].apply(list)

# Set the Date column as the index
df_monthly_vol.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_vol = df_monthly_vol['Betas'].apply(pd.Series)

# Set the index of df_monthly_vol to match df_48ind
df_monthly_vol.index = df_48ind.index

# Set the column names of df_monthly_vol to match df_48ind
df_monthly_vol.columns = df_48ind.columns


def select_extreme_values(row, num_values=5):
    sorted_row = row.sort_values(ascending=False)
    top_values = sorted_row.head(num_values)
    bottom_values = sorted_row.tail(num_values)
    return top_values, bottom_values


def get_returns(caracteristic, df_ret_shift, num_positions, weight_type='ew'):
    
    top_bottom_values = caracteristic.apply(select_extreme_values, axis=1)
    
    returns = []
    total_returns = []
    
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        
        if weight_type == 'ew':
            weight = 1.0 / num_positions
            top_returns = df_ret_shift.loc[date, top_indices] * weight
            bottom_returns = df_ret_shift.loc[date, bottom_indices] * weight * -1  # short positions have negative weight
        elif weight_type == 'vw':
            top_values = df_ret_shift.loc[date, top_indices]
            bottom_values = df_ret_shift.loc[date, bottom_indices]
            top_weights = top_values.abs() / top_values.abs().sum()
            bottom_weights = bottom_values.abs() / bottom_values.abs().sum()
            top_returns = top_values * top_weights
            bottom_returns = bottom_values * bottom_weights * -1  # short positions have negative weight
        else:
            raise ValueError("weight_type must be either 'ew' or 'vw'")
        
        returns.append((top_returns.sum(), bottom_returns.sum()))
    
    total_returns = [sum(x) for x in returns]

    return total_returns

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV'

In [115]:
import csv
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import gurobipy as gp
from gurobipy import GRB
from scipy.optimize import minimize
import matplotlib.dates as mdates
from datetime import datetime
import os
import statsmodels.api as sm
import warnings

In [84]:
def importClean_Ind(csv_file_path, desired_returns):
    """
    Cette fonction importe des données à partir d'un fichier CSV spécifié, effectue un nettoyage de données,
    et retourne un DataFrame contenant des données filtrées.

    Args:
        csv_file_path(str): Chemin vers le fichier CSV avec les données brutes.

    Return:
        DataFrame: DataFrame contenant les données historiques filtrées pour les 10 industries, 
        pour les 48 industries et certains facteurs de Fama-French.
    """


    # Importation
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        
        # Sauter les 10 premières lignes
        for _ in range(10):
            next(reader)

        # Lire le reste du fichier CSV dans un DataFrame
        df = pd.DataFrame(reader)
        df.columns = df.iloc[1]
        df = df.drop(df.index[1])
        df = df.reset_index(drop=True)

    # Nettoyage
    df = df.rename(columns={df.columns[0]: 'Date'})
    first_row = df[df.iloc[:, 0].str.strip() == desired_returns].index
    
    # Si 'desired_returns' est 'Number of Firms in Portfolios' ou 'Average Firm Size', 
    # alors sauter une ligne supplémentaire pour accéder aux données dans le bon format
    if desired_returns in ['Number of Firms in Portfolios', 'Average Firm Size', 'Sum of BE / Sum of ME']:
        df_AEWR_monthly = df.iloc[first_row.values[0] + 2:].reset_index(drop=True)
    else:
        df_AEWR_monthly = df.iloc[first_row.values[0] + 1:].reset_index(drop=True)
        
    last_row = (df_AEWR_monthly['Date'].str.len() != 6).idxmax()
    df_AEWR_monthly = df_AEWR_monthly.iloc[:last_row]
    
    # Adapter le format de la date en fonction de 'desired_returns'
    # Condition spécifique pour le facteur BtoM (Book-to-Market)
    if desired_returns in ['Sum of BE / Sum of ME']:  
        df_AEWR_monthly['Date'] = df_AEWR_monthly['Date'].str.strip()
        df_AEWR_monthly['Date'] = pd.to_datetime(df_AEWR_monthly['Date'], format='%Y')
    else:
        df_AEWR_monthly['Date'] = pd.to_datetime(df_AEWR_monthly['Date'], format='%Y%m')
    
    df_AEWR_monthly.iloc[:, 1:] = df_AEWR_monthly.iloc[:, 1:].astype(float)

    # Vérifier si le nom du fichier contient '48_Industry_Portfolios.CSV',
    # pour appliquer un nettoyage supplémentaire pour les NaN values
    if '48_Industry_Portfolios.CSV' in os.path.basename(csv_file_path):
        df_AEWR_monthly.replace(-99.99, np.nan, inplace=True)
        df_AEWR_monthly.replace(-999, np.nan, inplace=True)
        df_AEWR_monthly.dropna(inplace=True)
        
    # Retourner le DataFrame et fixer la 'Date' comme index  
    df = df_AEWR_monthly.set_index('Date', drop=True)
    
    return df

In [85]:
def importClean_rf(csv_file_path):
    """
    Cette fonction importe des données à partir du fichier CSV des 3 facteurs Fama-French, puis effectue un nettoyage de données
    et retourne un DataFrame contenant des données filtrées spécifiquement pour les taux sans risque dans la colonne 'RF'.

    Args:
        csv_file_path (str): Chemin vers le fichier CSV avec les données brutes.

    Return:
        DataFrame: DataFrame contenant les données nettoyées pour les taux sans risque.
    """
    
    
    # Importation des données en sautant les 3 premières lignes
    df = pd.read_csv(csv_file_path, skiprows=3)  

    # Renommage des colonnes pour faciliter la manipulation
    df.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

    # Suppression des lignes dont la date ne correspond pas au format '%Y%m'
    df = df[df['Date'].str.match(r'^\d{6}$', na=False)]

    # Conversion de 'Date' en datetime
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')

    # Conversion de 'RF' en float
    df['RF'] = df['RF'].astype(float)
    
    # Filtrage des colonnes 
    df = df[['Date', 'RF']] 
    
    # Mise en index de la colonne 'Date'
    df.set_index('Date', inplace=True)
    
    return df

## Functions.py

In [158]:
# Import data and clean
csv_file_path_10ind         = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie A/Data /48_Industry_Portfolios.CSV'
df_48Ind                    = importClean_Ind(csv_file_path_10ind, desired_returns='Average Value Weighted Returns -- Monthly')

csv_file_path_number_firm   = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie A/Data /48_Industry_Portfolios.CSV'
df_number_firm              = importClean_Ind(csv_file_path_number_firm, desired_returns='Number of Firms in Portfolios')
df_number_firm              = df_number_firm[df_number_firm.index >= df_48Ind.index.min()]

csv_file_path_avg_firm_size = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie A/Data /48_Industry_Portfolios.CSV'
df_average_firm_size        = importClean_Ind(csv_file_path_avg_firm_size, desired_returns='Average Firm Size')

csv_file_path_rf            = '/Users/thomasdeconinck/Documents/GitHub/Gestion de Portefeuille Devoir 2/Partie A/Data /F-F_Research_Data_Factors.CSV'
df_rf                       = importClean_rf(csv_file_path_rf)

# 2 Fama-French factors and Momentum factor data and calculation
# 1) SMB (Small Minus Big) : Market capitalization
market_caps                 = df_average_firm_size.multiply(df_number_firm)

# 2) HML (High Minus Low) : Book to Market ratio
csv_file_path_BtoM          = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie A/Data /48_Industry_Portfolios.CSV'
df_BtoM                     = importClean_Ind(csv_file_path_BtoM, desired_returns='Sum of BE / Sum of ME')
df_BtoM                     = df_BtoM.loc[df_BtoM.index.repeat(12)]
df_BtoM                     = df_BtoM.iloc[:len(df_48Ind)]
df_BtoM.index               = df_48Ind.index

# 3) UMD (Up Minus Down) : momentum (average of 12 months)
df_mom                      = df_48Ind.rolling(window=12, min_periods=12).mean()
df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)


# S'assurer que les données sont sur la même période de temps 
df_BtoM                    = df_BtoM.reindex(df_mom.index)
market_caps                = market_caps.reindex(df_mom.index)
df_48Ind                   = df_48Ind.reindex(df_mom.index)


# Rp,t+1 : Rendements des 48 industries à t+1 
df_ret_shift               = df_48Ind.shift(-1).dropna()

# Calculer le rendement annualisé du taux sans risque, selon la formule du taux de rendement annuel composé (CAGR)
mean_rf_monthly = df_FF_daily['RF'].mean()
mean_rf_annual = ((1 + mean_rf_monthly/100) ** 12 - 1)
mean_rf_annual*=100

In [172]:
def import_and_prepare_daily_data(csv_file_path):
    """
    Importe et prépare les données quotidiennes à partir d'un fichier CSV spécifié pour les 48 Industries 
    du portefeuille.

    Args:
    - csv_file_path (str): Le chemin complet vers le fichier CSV contenant les données quotidiennes 
    des rendements des 48 Industries.

    Return:
    - pd.DataFrame: Un DataFrame contenant les données quotidiennes nettoyées et préparées 
        pour les 48 Industries du portefeuille, avec les dates en index.
    """


    # Lecture du fichier CSV, avec l'en-tête à la ligne 6 (index 5)
    df_daily = pd.read_csv(csv_file_path, header=5)

    # Renommage de la première colonne pour indiquer qu'elle contient les dates
    df_daily = df_daily.rename(columns={'Unnamed: 0': 'Date'})

    # Copie et préparation des données jusqu'à la ligne spécifiée (25670)
    df_daily_ret = df_daily.iloc[:25670].copy()
    df_daily_ret['Date'] = pd.to_datetime(df_daily_ret['Date'], format='%Y%m%d')
    df_daily_ret.set_index('Date', inplace=True)
    df_daily_ret = df_daily_ret.apply(pd.to_numeric, errors='coerce')
    df_daily_ret.replace(-99.99, np.nan, inplace=True)
    df_daily_ret.replace(-999, np.nan, inplace=True)
    df_daily_ret.dropna(inplace=True)

    # Sélection d'un sous-ensemble des données à partir de la ligne spécifiée (232)
    df_daily_ret = df_daily_ret.iloc[231:].copy()

    return df_daily_ret


def import_and_clean_FF_3factors_daily(csv_file_path, df_reference):
    """
    Importe et nettoie les données des 3 quotidiennes des 3 facteurs Fama-French.

    Args:
    - csv_file_path (str): Chemin vers le fichier CSV contenant les données quotidiennes des facteurs Fama-French.
    - df_reference (pd.DataFrame): DataFrame de référence pour aligner les indices de dates, (df_daily_ret) dans cet exemple.

    Return:
        pd.DataFrame: DataFrame contenant les données quotidiennes des 3 facteurs Fama-French nettoyées et alignées.
    """


    # Lecture du fichier CSV, en supposant que l'en-tête se trouve à la 4ème ligne (index 3)
    df_FF = pd.read_csv(csv_file_path, header=3)

    # Renommage de la première colonne pour indiquer qu'elle contient les dates
    df_FF = df_FF.rename(columns={'Unnamed: 0': 'Date'})

    # Sélection des données jusqu'à la ligne spécifiée (25670 dans cet exemple)
    df_FF_daily = df_FF.iloc[:25670].copy()

    # Conversion de la colonne 'Date' au format datetime
    df_FF_daily['Date'] = pd.to_datetime(df_FF_daily['Date'], format='%Y%m%d')

    # Définition de la colonne 'Date' comme index du DataFrame
    df_FF_daily.set_index('Date', inplace=True)

    # Conversion des données en numérique, gestion des erreurs par coercition à NaN
    df_FF_daily = df_FF_daily.apply(pd.to_numeric, errors='coerce')

    # Remplacement des valeurs erronées ou manquantes spécifiques par NaN
    df_FF_daily.replace(-99.99, np.nan, inplace=True)
    df_FF_daily.replace(-999, np.nan, inplace=True)

    # Suppression des lignes contenant des NaN pour assurer la complétude des données
    df_FF_daily.dropna(inplace=True)

    # Alignement du DataFrame des facteurs Fama-French avec l'index du DataFrame de référence
    df_FF_daily_aligned = df_FF_daily.reindex(df_reference.index)

    return df_FF_daily_aligned


def import_FF_5factors_monthly(csv_file_path, df_reference):
    """
    Importe et prépare les données des 5 facteurs de Fama-French à partir d'un fichier CSV contenant les facteurs mensuels.

    Args:
    - csv_file_path (str): Le chemin complet vers le fichier CSV contenant les données des 5 facteurs de Fama-French.
    - df_reference (pd.DataFrame): DataFrame momemtum utilisé pour "locker" les données des 5 facteurs sur la base des dates de ce DataFrame.

    Returns:
        pd.DataFrame: DataFrame contenant les données des 5 facteurs de Fama-French nettoyées et préparées. 
    """


    # Lecture du fichier CSV, l'en-tête se trouve à la 2ème ligne (index 1)
    df_FF = pd.read_csv(csv_file_path, header=2)

    # Renommage de la première colonne 
    df_FF = df_FF.rename(columns={'Unnamed: 0': 'Date'})

    # Sélection des données pertinentes jusqu'à la ligne spécifiée (727)
    df_FF = df_FF.iloc[:727].copy()

    # Conversion de la colonne 'Date' en format datetime 
    df_FF['Date'] = pd.to_datetime(df_FF['Date'], format='%Y%m')

    # Définition de la colonne 'Date' comme index du DataFrame 
    df_FF.set_index('Date', inplace=True)

    # Conversion des valeurs du DataFrame en numérique, 
    # avec gestion des erreurs pour les valeurs non numériques
    df_FF = df_FF.apply(pd.to_numeric, errors='coerce')

    # Filtrage des données pour ne conserver que les dates présentes dans df_reference
    df_FF = df_FF.loc[df_reference.index]

    return df_FF



def calculate_betas(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']

    betas = []  # Create an empty list to store the betas

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = Rm_t

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        betas.append(results.params[1])  # Add the betas to the list

    # Convert the list of betas to a numpy array
    betas = np.array(betas)

    return betas


def calculate_monthly_betas(df_daily_ret, df_FF_daily, df_48ind):
    """
    Calcule les betas mensuels pour chaque mois à partir des données quotidiennes des rendements des 48 industries du portefeuille, 
    des facteurs Fama-French quotidiens et des 48 industries.

    Args:
    - df_daily_ret (pd.DataFrame): DataFrame contenant les rendements quotidiens.
    - df_FF_daily (pd.DataFrame): DataFrame contenant les facteurs quotidiens de Fama-French.
    - df_48ind (pd.DataFrame): DataFrame contenant des données pour les 48 industries, utilisé pour définir les indices et les noms des colonnes du résultat final.

    Returns:
    - pd.DataFrame: DataFrame contenant les betas mensuels pour chaque mois, avec les dates comme index et les industries comme colonnes.
    """
    
    
    # Obtenir la première et la dernière date dans les données
    first_date = df_daily_ret.index.min()
    last_date = df_daily_ret.index.max()

    # Créer une plage de dates pour chaque mois dans les données
    date_range = pd.date_range(start=first_date, end=last_date, freq='M')

    # Calculer les betas pour chaque mois
    monthly_betas = {}
    for date in date_range:
        start_date = date - pd.DateOffset(months=12)
        end_date = date
        betas = calculate_betas(start_date, end_date, df_daily_ret, df_FF_daily)  # Assurez-vous que la fonction `calculate_betas` est définie
        monthly_betas[date] = betas

    # Convertir le dictionnaire en DataFrame
    df_monthly_betas = pd.DataFrame(monthly_betas.items(), columns=['Date', 'Betas'])

    # Convertir la colonne 'Betas' de tableaux numpy en listes
    df_monthly_betas['Betas'] = df_monthly_betas['Betas'].apply(list)

    # Définir la colonne 'Date' comme index
    df_monthly_betas.set_index('Date', inplace=True)

    # Convertir chaque élément de la liste 'Betas' en une colonne distincte
    df_monthly_betas = df_monthly_betas['Betas'].apply(pd.Series)

    # Faire correspondre l'index de df_monthly_betas à df_48ind
    df_monthly_betas.index = df_48ind.index

    # Faire correspondre les noms des colonnes de df_monthly_betas à df_48ind
    df_monthly_betas.columns = df_48ind.columns

    return df_monthly_betas


def calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']
    SMB = df_FF_daily_selected['SMB']
    HML = df_FF_daily_selected['HML']

    volatilities = []  # Create an empty list to store the volatilities

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = pd.concat([Rm_t, SMB, HML], axis=1)

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        residuals = results.resid
        volatility = np.std(residuals)

        volatilities.append(volatility)  # Add the volatility to the list

    # Convert the list of volatilities to a numpy array
    volatilities = np.array(volatilities)

    return volatilities


def calculate_monthly_volatility(df_daily_ret, df_FF_daily_3factors_aligned, df_48Ind):
    """
    Calcule les volatilités idiosyncratiques mensuelles pour chaque mois à partir des données quotidiennes des rendements, 
    des facteurs Fama-French quotidiens alignés et des 48 industries.

    Args:
    - df_daily_ret_reshaped (pd.DataFrame): DataFrame contenant les rendements quotidiens reshaped.
    - df_daily_ret (pd.DataFrame): DataFrame contenant les rendements quotidiens.
    - df_FF_daily_3factors_aligned (pd.DataFrame): DataFrame contenant les facteurs quotidiens de Fama-French alignés.
    - df_48Ind (pd.DataFrame): DataFrame contenant des données pour les 48 industries.

    Returns:
    - pd.DataFrame: DataFrame contenant les volatilités idiosyncratiques mensuelles pour chaque mois, 
                    avec les dates comme index et les industries comme colonnes.
    """
    
    
    # Get the first and last date in the data
    first_date = df_daily_ret.index.min()
    last_date = df_daily_ret.index.max()

    # Create a date range for each month in the data
    date_range = pd.date_range(start=first_date, end=last_date, freq='M')

    # Calculate the volatilities for each month
    monthly_volatilities = {}
    for date in date_range:
        start_date = date - pd.DateOffset(months=1) + pd.DateOffset(days=1)
        end_date = date
        volatilities = calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret, df_FF_daily_3factors_aligned)
        monthly_volatilities[date] = volatilities

    # Convert the dictionary to a DataFrame
    df_monthly_vol = pd.DataFrame(monthly_volatilities.items(), columns=['Date', 'Betas'])

    # Convert the Betas column from numpy arrays to lists
    df_monthly_vol['Betas'] = df_monthly_vol['Betas'].apply(list)

    # Set the Date column as the index
    df_monthly_vol.set_index('Date', inplace=True)

    # Convert each item in the 'Betas' list to a separate column
    df_monthly_vol = df_monthly_vol['Betas'].apply(pd.Series)

    # Set the index of df_monthly_vol to match df_48Ind
    df_monthly_vol.index = df_48Ind.index

    # Set the column names of df_monthly_vol to match df_48Ind
    df_monthly_vol.columns = df_48Ind.columns

    return df_monthly_vol


def select_extreme_values(row, num_values=5):
    """
    Sélectionne les valeurs extrêmes dans une ligne (série) en fonction des valeurs triées dans l'ordre décroissant, 
    et retourne les premières valeurs les plus élevées et les dernières les plus basses, pour une caractéristique donnée.

    Args:
    - row (pd.Series): Une ligne (série) contenant les valeurs des caractéristiques à trier.
    - num_values (int, optional): Le nombre d'industries avec la caractéristique extrêmes à sélectionner. Fixé à 5 par défaut.

    Returns:
    - tuple: Un tuple contenant deux pandas Series, les premières "num_values" valeurs les plus élevées 
            et les dernières "num_values" valeurs les plus basses pour chaque caractéristique donnée.
    """
    
    
    # Triage dans l'ordre décroissant des caractéristiques (plus grande à plus petite) 
    sorted_row = row.sort_values(ascending=False)
    
    # Sélection des premières "num_values" valeurs les plus élevées
    top_values = sorted_row.head(num_values)
    
    # Sélection des dernières "num_values" valeurs les plus basses
    bottom_values = sorted_row.tail(num_values)
    
    return top_values, bottom_values


def get_returns(caracteristic, df_ret_shift, num_positions, weight_type='ew'):
    """
    Calcule les rendements totaux pour les positions longues et courtes basées sur les caractéristiques données, 
    les rendements des industries et le nombre de positions à prendre.

    Args:
    - caracteristic (pd.DataFrame): DataFrame contenant les caractéristiques de chaque industrie, 
                                    avec les dates en index et les industries en colonnes.
    - df_ret_shift (pd.DataFrame): DataFrame contenant les rendements des industries, avec un décalage de t+1 (shifted by 1), 
                                    pour calculer les rendements totaux pour les positions longues et courtes.
    - num_positions (int): Nombre de positions à prendre (par exemple, 5 industries extrêmes de chaque côté pour un total de 10 positions à chaque fois).
    - weight_type (str, optional): Type de pondération à utiliser, 'ew' pour équipondéré ou 'vw' pour pondération par capitalisation boursière (marketcap).
                                    Par défaut, 'ew', équipondéré.

    Returns:
    - list: Une liste contenant les rendements totaux pour les positions longues et courtes à la fin de chaque mois t+1. 
    """
    
    
    # Sélectionne les valeurs extrêmes pour chaque mois t 
    top_bottom_values = caracteristic.apply(select_extreme_values, axis=1)
    
    returns = []  # Initialise une liste pour stocker les rendements individuels pour chaque mois
    total_returns = []  # Initialise une liste pour stocker les rendements totaux pour chaque mois
    
    # Itération à travers toutes les valeurs extrêmes pour chaque mois 
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        
        # Calcul des rendements pour les positions longues et courtes en fonction du type de pondération spécifié 
        if weight_type == 'ew':
            weight = 1.0 / num_positions  # Poids égal pour chaque position
            top_returns = df_ret_shift.loc[date, top_indices] * weight
            bottom_returns = df_ret_shift.loc[date, bottom_indices] * weight * -1  # Les positions courtes ont un poids négatif
        elif weight_type == 'vw':
            top_values = df_ret_shift.loc[date, top_indices]
            bottom_values = df_ret_shift.loc[date, bottom_indices]
            top_weights = top_values.abs() / top_values.abs().sum()  # Pondération par capitalisation boursière absolue 
            bottom_weights = bottom_values.abs() / bottom_values.abs().sum()  # Pondération par capitalisation boursière absolue 
            top_returns = top_values * top_weights 
            bottom_returns = bottom_values * bottom_weights * -1  # Les positions courtes ont un poids négatif 
        else:
            raise ValueError("weight_type must be either 'ew' or 'vw'")
        
        # Ajoute les rendements longs et courts à la liste des rendements
        returns.append((top_returns.sum(), bottom_returns.sum()))  
    
    # Calcule les rendements totaux pour chaque mois
    total_returns = [sum(x) for x in returns]  

    return total_returns


def get_sharpe_and_constants(caracteristic, df_ret_shift, num_positions, df_FF_daily_3factors_aligned, df_FF_5, weight_type='ew'):
    """
    Calcule le ratio de Sharpe et les constantes de régression (Alpha de Jensen) pour les rendements totaux des stratégies 'EW' et 'VW' en fonction des caractéristiques données,
    des rendements journaliers des 48 industries, du nombre de positions longues et courtes à prendre, du taux sans risque moyen annualisé selon la formule du (CAGR) et des 3, 4 et 5 facteurs de Fama-French.

    Args:
    - caracteristic (pd.DataFrame): DataFrame contenant les caractéristiques de chaque industrie, 
                                    avec les dates en index et les industries en colonnes.
    - df_ret_shift (pd.DataFrame): DataFrame contenant les rendements des industries, avec un décalage de t+1, 
                                    pour calculer les rendements totaux pour les positions longues et courtes.
    - num_positions (int): Nombre de positions à prendre (par exemple, 5 industries extrêmes de chaque côté pour un total de 10 positions à chaque fois).
    - rf (float): Taux sans risque mensuel moyen sur la période.
    - df_FF_5 (pd.DataFrame): DataFrame contenant les 5 facteurs de Fama-French, 
                                utilisés dans la régression pour calculer les constantes.
    - weight_type (str, optional): Type de pondération à utiliser, 'ew' pour équipondéré ou 'vw' pour pondération par capitalisation boursière.
                                    Par défaut, 'ew', équipondéré.

    Returns:
    - tuple: Un tuple contenant le ratio de Sharpe et les constantes de régression pour 3, 4 et 5 facteurs.
    """
    # Calculer le rendement annualisé du taux sans risque, selon la formule du taux de rendement annuel composé (CAGR)
    mean_rf_monthly = df_FF_daily_3factors_aligned['RF'].mean()
    mean_rf_annual = ((1 + mean_rf_monthly/100) ** 12 - 1)
    mean_rf_annual*=100

    # Obtenir les rendements totaux pour les positions longues et courtes 
    # en fonction du triage des caractéristiques et du type de pondération
    total_returns = get_returns(caracteristic, df_ret_shift, num_positions, weight_type)

    # Convertir la liste en pandas Series 
    total_returns_series = pd.Series(total_returns)
    
    # Calculer la moyenne et l'écart-type des rendements mensuels du portefeuille pour la période indiquée
    mean_return_monthly = total_returns_series.mean()
    std_return_monthly = total_returns_series.std()

    # Annualiser la moyenne et l'écart-type des rendements mensuels, avec la formule du taux de rendement annuel composé (CAGR)
    mean_return_annualized = (1 + mean_return_monthly/100) ** 12 - 1 
    std_return_annualized = std_return_monthly * np.sqrt(12) 

    # Convertir en pourcentage pour faciliter les manipulations par la suite
    mean_return_annualized = mean_return_annualized * 100 

    # Calculer le ratio de Sharpe pour les rendements totaux 
    sharpe_ratio = (mean_return_annualized - mean_rf_annual) / std_return_annualized

    # Convertir la liste en pandas DataFrame pour la régression pour obtenir les alpha de Jensen 
    total_returns_df = pd.DataFrame(total_returns, columns=['returns'])

    # Réinitialiser l'index pour les deux DataFrames 
    total_returns_df = total_returns_df.reset_index(drop=True)
    df_FF_5 = df_FF_5.reset_index(drop=True)

    # Régression avec 3, 4 et 5 facteurs de Fama-French pour obtenir les constantes (Alpha de Jensen)
    constants = []
    for num_factors in [3, 4, 5]:
        factors = sm.add_constant(df_FF_5.iloc[:, :num_factors])
        model = sm.OLS(total_returns_df, factors)
        results = model.fit()
        constants.append(results.params['const'])

    # Retourner le ratio de Sharpe et les constantes de régression pour 3, 4 et 5 facteurs 
    return sharpe_ratio, constants

## TP2 Notebook Partie B.ipynb

In [176]:
# Ignorer les avertissements de type FutureWarning et DtypeWarning pour une meilleure lisibilité des résultats
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

csv_file_path_daily_ret = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie B/Data/48_Industry_Portfolios_Daily.csv' 
desired_returns = "Average Value Weighted Returns -- Daily"

df_48Ind_daily = import_and_prepare_daily_data(csv_file_path_daily_ret)


csv_file_path_FF_daily_factors = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie B/Data/F-F_Research_Data_Factors_daily.CSV'
df_FF_daily_3factors_aligned = import_and_clean_FF_3factors_daily(csv_file_path_FF_daily_factors, df_48Ind_daily)


csv_file_path = '/Users/thomasdeconinck/Downloads/Gestion-de-Portefeuille-Devoir-2/Partie B/Data/F-F_Research_Data_5_Factors_2x3-2.csv'
df_FF_5 = import_FF_5factors_monthly(csv_file_path, df_mom)


# Cacluler les betas mensuels avec la fonction calculate_monthly_betas 
df_monthly_betas = calculate_monthly_betas(df_48Ind_daily, df_FF_daily_3factors_aligned, df_48Ind)

# Calculer les volatilités idiosyncratiques mensuelles avec la fonction calculate_monthly_volatility
df_monthly_vol = calculate_monthly_volatility(df_48Ind_daily, df_FF_daily_3factors_aligned, df_48Ind)

dataframes = [market_caps, df_BtoM, df_mom, df_monthly_betas, df_monthly_vol]
names = ['df_MC', 'df_BtoM', 'df_mom', 'df_monthly_betas', 'df_monthly_vol']
date_ranges = [('1970-06-01', '2024-01-01'), ('1990-01-01', '2024-01-01'), ('2000-01-01', '2024-01-01')]

for df, name in zip(dataframes, names):
    for start_date, end_date in date_ranges:
        # Filter the DataFrame for the given date range
        df_filtered = df.loc[start_date:end_date]
        df_FF_5_filtered = df_FF_5.loc[start_date:end_date]

        # Calculate Sharpe ratio and constants
        sharpe_ratio_ew, constants_ew = get_sharpe_and_constants(df_filtered, df_48Ind, 5, df_FF_daily_3factors_aligned, df_FF_5_filtered, 'ew')
        sharpe_ratio_vw, constants_vw = get_sharpe_and_constants(df_filtered, df_48Ind, 5, df_FF_daily_3factors_aligned, df_FF_5_filtered, 'vw')

        # Print results
        print(f"For {name} from {start_date} to {end_date}:")
        print("EW Sharpe Ratio: ", sharpe_ratio_ew)
        print("EW Constants: ", constants_ew)
        print("VW Sharpe Ratio: ", sharpe_ratio_vw)
        print("VW Constants: ", constants_vw)
        print("\n")

For df_MC from 1970-06-01 to 2024-01-01:
EW Sharpe Ratio:  -0.12789708199111574
EW Constants:  [0.1744608932148895, 0.3631301856404769, 0.41686805413367467]
VW Sharpe Ratio:  -0.25597625339191654
VW Constants:  [-0.16234627403842689, 0.0926170589524268, 0.18231550886637185]


For df_MC from 1990-01-01 to 2024-01-01:
EW Sharpe Ratio:  -0.08327833605752168
EW Constants:  [0.08498345574364319, 0.31321890666469904, 0.378340529031491]
VW Sharpe Ratio:  -0.21151211574727447
VW Constants:  [-0.3105169510924924, 0.03059911342345918, 0.1520129094260069]


For df_MC from 2000-01-01 to 2024-01-01:
EW Sharpe Ratio:  -0.3141243425907897
EW Constants:  [-0.20212459364281937, 0.049627136756509586, 0.13339816783416042]
VW Sharpe Ratio:  -0.3520490044420156
VW Constants:  [-0.6129138022701308, -0.1922420594009704, -0.02803668908266575]


For df_BtoM from 1970-06-01 to 2024-01-01:
EW Sharpe Ratio:  0.18399542860406892
EW Constants:  [-0.14928438271854022, -0.11082226432462888, -0.08640059206254175]
VW S

In [91]:

########## boucle pour toutes les colonnes, avec une date de début et de fin pour chaque mois:

def get_sharpe_and_constants(caracteristic, df_ret_shift, num_positions, rf, df_FF_5, weight_type='ew'):
    total_returns = get_returns(caracteristic, df_ret_shift, num_positions, weight_type)

    # Convert list to pandas Series
    total_returns_series = pd.Series(total_returns)

    # Calculate Sharpe ratio
    sharpe_ratio = ((total_returns_series.mean() - rf) / total_returns_series.std()) * np.sqrt(12)

    # Convert list to pandas DataFrame
    total_returns_df = pd.DataFrame(total_returns, columns=['returns'])

    # Reset index
    total_returns_df = total_returns_df.reset_index(drop=True)
    df_FF_5 = df_FF_5.reset_index(drop=True)

    # Regression with 3, 4, and 5 factors
    constants = []
    for num_factors in [3, 4, 5]:
        factors = sm.add_constant(df_FF_5.iloc[:, :num_factors])
        model = sm.OLS(total_returns_df, factors)
        results = model.fit()
        constants.append(results.params['const'])

    return sharpe_ratio, constants

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV'

In [ ]:
dataframes = [df_MC, df_BtoM, df_mom, df_monthly_betas, df_monthly_vol]
names = ['df_MC', 'df_BtoM', 'df_mom', 'df_monthly_betas', 'df_monthly_vol']
date_ranges = [('1970-06-01', '2024-01-01'), ('1990-01-01', '2024-01-01'), ('2000-01-01', '2024-01-01')]

for df, name in zip(dataframes, names):
    for start_date, end_date in date_ranges:
        # Filter the DataFrame for the given date range
        df_filtered = df.loc[start_date:end_date]
        df_FF_5_filtered = df_FF_5.loc[start_date:end_date]

        # Calculate Sharpe ratio and constants
        sharpe_ratio_ew, constants_ew = get_sharpe_and_constants(df_filtered, df_48ind, 5, rf, df_FF_5_filtered, 'ew')
        sharpe_ratio_vw, constants_vw = get_sharpe_and_constants(df_filtered, df_48ind, 5, rf, df_FF_5_filtered, 'vw')

        # Print results
        print(f"For {name} from {start_date} to {end_date}:")
        print("EW Sharpe Ratio: ", sharpe_ratio_ew)
        print("EW Constants: ", constants_ew)
        print("VW Sharpe Ratio: ", sharpe_ratio_vw)
        print("VW Constants: ", constants_vw)
        print("\n")

For df_MC from 1970-06-01 to 2024-01-01:
EW Sharpe Ratio:  -0.12903594319668069
EW Constants:  [0.1744608932148895, 0.36313018564047694, 0.41686805413367434]
VW Sharpe Ratio:  -0.2638945743872925
VW Constants:  [-0.1623462740384269, 0.09261705895242703, 0.1823155088663715]


For df_MC from 1990-01-01 to 2024-01-01:
EW Sharpe Ratio:  -0.08380310703698962
EW Constants:  [0.08498345574364308, 0.31321890666469904, 0.37834052903149046]
VW Sharpe Ratio:  -0.21754452007960087
VW Constants:  [-0.3105169510924926, 0.030599113423459178, 0.15201290942600637]


For df_MC from 2000-01-01 to 2024-01-01:
EW Sharpe Ratio:  -0.32265847026153704
EW Constants:  [-0.20212459364281926, 0.04962713675650925, 0.13339816783416092]
VW Sharpe Ratio:  -0.370356563473417
VW Constants:  [-0.6129138022701306, -0.19224205940097078, -0.028036689082665513]


For df_BtoM from 1970-06-01 to 2024-01-01:
EW Sharpe Ratio:  -0.31282307408555426
EW Constants:  [-0.8677918194689576, -0.8073048677898551, -0.725928504649743]
VW 